In [1]:
import pyspark
from pyspark import SparkContext, SQLContext
sc = pyspark.SparkContext('local[*]')
sqlc = SQLContext(sc)

In [2]:
from pyspark.sql import functions as F, Window
from pyspark.sql.functions import *
from pyspark.ml.recommendation import ALSModel
from pyspark.sql import Row
from pyspark.sql.functions import desc


In [4]:
import json
import sqlite3

### Load Data for als

In [8]:
file_path=!pwd

In [11]:
users=sqlc.read.json(file_path[0]+'/data/users.json')
als_data=sqlc.read.json(file_path[0]+'/data/forks_stars_sample_prepared.json')
repos=sqlc.read.json(file_path[0]+'/data/projects_not_forked_2017.json')


In [7]:
als_data.select('user_id').distinct().count()

24225

In [13]:
pred_ur=sqlc.read.json(file_path[0]+'/data/forks_stars_sample_ur_predictions_train.json')


In [9]:
pred_ur.select('user_id').distinct().count()

24225

In [15]:
model=ALSModel.load(file_path[0]+'/data/als_sample_r10_i10_reg01_fs.parquet')

In [16]:
als_data=als_data.select('repo_id','repo_idn','user_id','user_idn')

In [17]:
repos=repos.select(col('id').alias('repo_id'),'owner_id','name')

In [18]:
repos=repos.join(users.select('id',col('login').alias('owner')),users.id==repos.owner_id,'left')

### Find info about users and repositories contained in als data

In [19]:
als_data

DataFrame[user_id: string, repo_id: string, repo_idn: double, user_idn: double, login: string]

In [10]:
als_data=als_data.join(users.select(col('id').alias('user_id'),'login'),'user_id','inner')
als_repos=repos.join(als_data.select('repo_id','repo_idn').distinct(),'repo_id','inner') #repos from the model#

In [20]:
als_data.count()

652596

In [44]:
als_repos.count()

1269

In [45]:
als_data.select('repo_id').distinct().count()

1269

In [111]:
als_repos.show()

+--------+--------+--------------------+--------+-----------------+--------+
| repo_id|owner_id|                name|      id|            owner|repo_idn|
+--------+--------+--------------------+--------+-----------------+--------+
|53840452|  177657|         caddy-blink|  177657|            mholt| 99589.0|
|53842956|31145670|DDDUR2016F_514202...|31145670|       skyfalls17|190141.0|
|53847115|31161344|           StockTech|31161344|csxiaoyaojianxian| 56870.0|
|53850999| 6138279|             restapi| 6138279|            alesr|260826.0|
|53854062| 1269595|               phell| 1269595|           gale93|179341.0|
|53860467|    5441|                echo|    5441|  kelseyhightower|  7911.0|
|53875996| 2654229|    swing-ble-tester| 2654229|         yenchieh|202871.0|
|53896628|33246187|               shard|33246187|         ShardApp| 79958.0|
|53920311| 1267544| GoogleCalendarSkill| 1267544|          jcasoft|204892.0|
|53930706|33162225|             website|33162225| factcheckthenews|234500.0|

## Recommendations for ALS

### Easy way

In [20]:
als_recs=model.recommendForAllUsers(30)
als_recs=als_recs.withColumn('scores',explode('recommendations'))
als_recs=als_recs.withColumn('repo_idn',als_recs.scores['repo_idn'])\
                           .withColumn('score',als_recs.scores['rating'])

In [21]:
# find users and repo infos

als_recs=als_recs.select('user_idn','repo_idn','score').join(als_data.select('user_idn','login').distinct(),'user_idn','inner')

In [22]:
als_recs=als_recs.join(als_repos.select('repo_idn','name'),'repo_idn','left')

In [23]:
als_recs.filter("login='iver56'").show(30)

+--------+--------+----------+------+--------------------+
|repo_idn|user_idn|     score| login|                name|
+--------+--------+----------+------+--------------------+
|       8|   15972| 0.9017224|iver56|API-Security-Chec...|
|     442|   15972|  0.901715|iver56|devportfolio-temp...|
|     116|   15972| 0.9017494|iver56|      node-interview|
|      29|   15972| 0.9017723|iver56|hiring-without-wh...|
|      64|   15972|0.90172666|iver56|        tamperchrome|
|      44|   15972|0.90173244|iver56|digital_video_int...|
|      35|   15972|  0.901804|iver56| lottie-react-native|
|      96|   15972| 0.9017681|iver56|             seq2seq|
|    1157|   15972| 0.9017875|iver56|        awesome-bots|
|       1|   15972|   0.90179|iver56|Best-websites-a-p...|
|     408|   15972| 0.9017996|iver56|   react-native-node|
|     421|   15972|  0.901723|iver56|    vue-recyclerview|
|     833|   15972| 0.9017478|iver56|career-developmen...|
|     435|   15972| 0.9017252|iver56|                gai

### Alternative way

In [11]:
def transform_to_json(row):
    return {row.repo_idn :{'name':row.name,'owner': row.owner}}

list_repos=als_repos.select('repo_idn','name','owner').rdd.map(transform_to_json).collect()

dict_repos={k:v for d in list_repos for k,v in d.items()}


In [42]:
def give_rec(df,username,n):
    user=df.filter(df.login==username).select(df.user_idn).rdd.map(lambda x: x.user_idn).collect()[0]
    already_fs=df.filter(df.login==username).select(df.repo_idn).rdd.map(lambda x: x.repo_idn).collect()
    new_repos=list(set(list(dict_repos.keys()))-set(already_fs))
    user_rows = [Row(user_idn=user, 
                 repo_idn=float(i)) for i in new_repos]
    user_df = sqlc.createDataFrame(user_rows)
    user_predictions=model.transform(user_df).na.fill(-5.0)
    top_predictions=user_predictions.sort('prediction',ascending=False).select('repo_idn','prediction')\
                                    .rdd.map(lambda x: (x.repo_idn,x.prediction)).take(n)
    links=[]
    scores=[]
    for x,y in top_predictions:
        link=r'https://github.com/'+dict_repos[x]['owner']+'/'+dict_repos[x]['name']
        links.append(link)
        scores.append('%.4f' % y)
    d_links=dict()
    d_scores=dict()
    d_links['links']=links
    d_scores['scores']=scores
    return [json.dumps(d_links),json.dumps(d_scores)]                       
    

In [43]:
give_rec(als_data,username='michcioperz',n=5)

['{"links": ["https://github.com/bmorelli25/Become-A-Full-Stack-Web-Developer", "https://github.com/zeeshanu/learn-regex", "https://github.com/airbnb/lottie-react-native", "https://github.com/STALTZ/react-native-node", "https://github.com/sdmg15/Best-websites-a-programmer-should-visit"]}',
 '{"scores": ["0.9019", "0.9018", "0.9018", "0.9018", "0.9018"]}']

### Find info about users and repositories in pred_ur

In [25]:
pred_ur=pred_ur.withColumn('scores',explode('rec'))
pred_ur=pred_ur.withColumn('repo_id',pred_ur.scores['item'])\
                       .withColumn('score',pred_ur.scores['score'])

pred_ur=pred_ur.select('user_id','repo_id','score')

In [26]:
pred_ur=pred_ur.join(users.select(col('id').alias('user_id'),'login'),'user_id','inner')
pred_ur=pred_ur.join(repos.select('repo_id','name','owner'),'repo_id','left')


In [27]:
pred_ur.cache()

DataFrame[repo_id: string, user_id: string, score: double, login: string, name: string, owner: string]

In [28]:
als_data.cache()

DataFrame[user_id: string, repo_id: string, repo_idn: double, user_idn: double, login: string]

In [29]:
w=(Window.partitionBy('user_id').orderBy(col('score').desc()).rowsBetween(Window.unboundedPreceding, Window.currentRow))
pred_ur=pred_ur.withColumn('recs',F.count('user_id').over(w))

In [30]:
pred_ur.show()

+--------+--------+-------------------+--------+--------------------+-------------+----+
| repo_id| user_id|              score|   login|                name|        owner|recs|
+--------+--------+-------------------+--------+--------------------+-------------+----+
|68785107|10013904|  1.570141077041626|morganel|CarND-Semantic-Se...|      udacity|   1|
|61798405|10013904|  1.183546781539917|morganel|           aind2-cnn|      udacity|   2|
|65481637|10013904|  1.144757866859436|morganel|RoboND-Rover-Project|      udacity|   3|
|64535847|10013904| 1.1407822370529175|morganel|   CarND-MPC-Quizzes|      udacity|   4|
|64253797|10013904| 1.1407822370529175|morganel|            aind2-dl|      udacity|   5|
|57507098|10013904|  1.117347240447998|morganel|AIND-Simulated_An...|      udacity|   6|
|54768394|10013904|  1.117347240447998|morganel|         aind-sudoku|      udacity|   7|
|65200326|10013904|  1.117347240447998|morganel|       AIND-CV-Mimic|      udacity|   8|
|55936096|10013904| 1

### Get recommendations for ur

In [44]:
def ur_give_rec(df,username,n):
    user_repos=df.filter(df.login==username).filter(df.recs<=n).select('name','owner')\
                                            .rdd.map(lambda x: (x.name,x.owner)).collect()
    user_scores=df.filter(df.login==username).filter(df.recs<=n).select('score')\
                                             .rdd.map(lambda x: x.score).collect()
    d_links=dict()
    d_scores=dict()
    d_links['links']=[r'https://github.com/'+y+'/'+x for (x,y) in user_repos]
    d_scores['scores']=['%.4f' % score for score in user_scores]
    return [json.dumps(d_links),json.dumps(d_scores)] 

In [45]:
give_rec(als_data,username='michcioperz',n=5)

['{"links": ["https://github.com/bmorelli25/Become-A-Full-Stack-Web-Developer", "https://github.com/zeeshanu/learn-regex", "https://github.com/airbnb/lottie-react-native", "https://github.com/STALTZ/react-native-node", "https://github.com/sdmg15/Best-websites-a-programmer-should-visit"]}',
 '{"scores": ["0.9019", "0.9018", "0.9018", "0.9018", "0.9018"]}']

In [46]:
ur_give_rec(pred_ur,username='michcioperz',n=5)

['{"links": ["https://github.com/k4m4/terminals-are-sexy", "https://github.com/aguinet/wannakey", "https://github.com/gentilkiwi/wanakiwi", "https://github.com/s0lst1c3/eaphammer", "https://github.com/GoogleChrome/puppeteer"]}',
 '{"scores": ["0.1941", "0.1905", "0.1893", "0.1585", "0.1452"]}']

### Save results in a SQlite db

In [47]:
users_sample=als_data.select('login').distinct().limit(10).rdd.map(lambda x: x.login).collect()

In [76]:
users_sample

['michcioperz',
 'iver56',
 'T-Rex',
 'skyleronken',
 'amaanite890',
 'andyeskridge',
 'miofthena',
 'wrahim',
 'Xu-TikT',
 'jescalan']

In [48]:
conn = sqlite3.connect('database_reccom.db')
c = conn.cursor()


In [49]:
c.execute('''DROP TABLE als_rec''')

In [50]:
c.execute('''CREATE TABLE als_rec (name text, rec text, scores text)''')

In [52]:
for item in users_sample:
    c.execute("INSERT INTO als_rec VALUES ('%s','%s', '%s')" % (item,give_rec(als_data,item,n=5)[0],give_rec(als_data,item,n=5)[1]))

In [54]:
c.execute('''DROP TABLE ur_rec''')

In [55]:
c.execute('''CREATE TABLE ur_rec (name text, rec text, scores text)''')

In [56]:
for item in users_sample:
    c.execute("INSERT INTO ur_rec VALUES ('%s','%s', '%s')" % (item,ur_give_rec(pred_ur,item,n=5)[0],ur_give_rec(pred_ur,item,n=5)[1]))
    



In [ ]:
t=('',)

In [73]:
for x,y,z in c.execute('SELECT * FROM als_rec_sample WHERE name=?',t):
    print(x)
    print(y)
    print(z)

In [57]:
for row in c.execute("SELECT * FROM ur_rec"):
    print(row)

('michcioperz', '{"links": ["https://github.com/k4m4/terminals-are-sexy", "https://github.com/aguinet/wannakey", "https://github.com/gentilkiwi/wanakiwi", "https://github.com/s0lst1c3/eaphammer", "https://github.com/GoogleChrome/puppeteer"]}', '{"scores": ["0.1941", "0.1905", "0.1893", "0.1585", "0.1452"]}')
('iver56', '{"links": ["https://github.com/dhammack/DSB2017", "https://github.com/caocuong0306/awesome-object-proposals", "https://github.com/juliandewit/kaggle_ndsb2017", "https://github.com/YadiraF/GAN", "https://github.com/hunkim/DeepLearningZeroToAll"]}', '{"scores": ["0.5218", "0.5215", "0.5086", "0.4288", "0.4276"]}')
('T-Rex', '{"links": ["https://github.com/laotan7237/EasyReader", "https://github.com/andremion/Louvre", "https://github.com/googlesamples/android-architecture-components", "https://github.com/zhangyuChen1991/Roll3DImageView", "https://github.com/yarolegovich/SlidingRootNav"]}', '{"scores": ["0.4860", "0.4347", "0.3984", "0.3536", "0.3202"]}')
('skyleronken', '{

In [58]:
conn.commit()
conn.close()

In [19]:
sc.stop()